# Защита персональных данных клиентов

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

Нам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуем корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [2]:
insurance = pd.read_csv('C:\\Users\\503so\\OneDrive\\Desktop\\praktikum-to-git\\10_insurance.csv')

In [3]:
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
insurance.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [5]:
insurance.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Наш датасет содержит записи о 5000 объектов без пропусков в числовом формате. Категориальный признак один - пол: мужской и женский представлены в виде 1 и 0 (мы не знаем, какому полу соответствует какое обозначение).
Оставшиеся 4 признака - количественные: возраст в годах, зарплата в рублях, количество членов семьи и количество удовлетворенных обращений по страховым выплатам. Последний признак - целевой.

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

In [6]:
#выделим из исходных данных фичи и целевой признак
features = insurance.drop('Страховые выплаты', axis=1)
target = insurance['Страховые выплаты']

In [7]:
#преобразуем фичи в матрицу с дополнительным нулевым столбцом, заполненным единицами
X = np.concatenate((np.ones((features.shape[0], 1)), features), axis=1)

In [8]:
#преобразцем целевой признак в вектор
y = target.values

In [9]:
#выполним расчеты согласно формуле
w = np.linalg.inv(X.T @ X) @ X.T @ y

In [10]:
#получим предсказания (переменная а)
predictions = X @ w

In [11]:
#вычислим коэффициент детерминации предсказаний по отношению к целевому признаку
r2_original = r2_score(target, predictions)

In [12]:
print(r2_original)

0.42494550286668


## Алгоритм преобразования

**Алгоритм**
1. Создать квадратную обратимую матрицу размерности 4х4 (по количеству признаков), заполнить ее случайными числами.
2. Выполнить операцию матричного умножения матрицы признаков на полученную квадратную матрицу.

Таким образом, за 2 шага мы выполним шифрование исходных данных.


**Обоснование**

Для того, чтобы измененные показатели признаков дали на выходе модели корректный результат, необходимо, чтобы данные подвергались изменениям "пропорционально", не нарушались взаимосвязи векторов признаков.

Для этой задачи нам подходит умножение исходной матрицы слева на обратимую матрицу размерности количества признаков. Фактически, умножение на обратимую матрицу - трансформирование исходных данных в виде растяжения и поворота исходных векторов относительно начала координат.

В результате выполнения алгоритма мы должны получить предсказания на основании измененных данных, равные предсказаниям на исходных данных:

$$ a = a'$$

где:
- **а** - предсказания на исходных данных;
- **а** - предсказания на измененных данных.

Для этого воспользуемся формулой вычисления предсказаний:

$$
a = X*w
$$

Поскольку мы трансформируем данные умножением на обратимую матрицу **P**, формулу предсказаний по измененным данным можно записать в следующем виде:

$$
a' = X*P*w'
$$

Обратимся к формуле сдвига:

$$
w = (X^T*X)^{-1}*X^T*y
$$
Для простоты примем, что

$$
X*P = X'
$$
тогда

$$
a' = X'*w'
$$

$$
w' = (X'^T*X')^{-1}*(X')^T*y
$$

Подставим значение **w'** в формулу **a'**:

$$ 
a' = X'*(X'^T*X')^{-1}*(X')^T*y
$$

$$
a' = X'*X'^{-1}*(X'^T)^{-1}*(X')^T*y
$$

Поскольку 
$$
X*X^{-1}=X^{-1}*X=E
$$

Формула **a'** примет вид

$$
a' = E*E*y
$$

Т.к. **E** - единичная матрица, **a'** зависит только от **y**. А поскольку **y** - целевой признак, не подвергающийся трансформации, можем заключить, что на **a'** не повлияет трансформация исходной матрицы.

Т.к. размерность **X** - $(m*n)$, размерность **P** - $(n*n)$, матрица $X*P$ будет иметь размерность, равную размерности **Х** - $(m*n)$.

Следовательно, единичная матрица $E'=(X'*X'^{-1})$ будет иметь ту же размерность, что и $E=(X*X^{-1})$

Т.е.

$$
E*E*y = E'*E'*y
$$

или 

$$ a = a'$$

Что и требовалось доказать.


Аналогичное доказательство можно было привести на исходной матрице **Х**, в данном случае выбор исходных или измененных данных для доказательства не принципиален.

## Проверка алгоритма

In [13]:
def square_matrix(m):
    matrix = np.random.normal(size = [m, m])
    if (np.dot(matrix , np.linalg.inv(matrix))).all == np.eye(m).all:
           result = matrix
    else:
        result = np.random.normal(size = [m, m])
    return result

In [14]:
P = square_matrix(4)
if np.linalg.det(P) != 0:
    print('Матрица обратима, определитель существует и не равнен нулю')
else:
    print('Матрица необратима. Необходимо перезапустить ячейку и сгенерировать матрицу повторно')

Матрица обратима, определитель существует и не равнен нулю


In [15]:
transform_features =  features.values @ P

In [16]:
#проверим размерность полученной матрицы
transform_features.shape

(5000, 4)

In [17]:
model = LinearRegression()

In [18]:
#выполним разделение фичей и целевого признака на обучающую и тестовую выборки
features_train, features_test, target_train, target_test = train_test_split(
features, target, test_size=.25, random_state=503)

In [19]:
#обучим модель
model.fit(features_train, target_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [20]:
#получим предсказания модели на тестовых признаках
predictions_t = model.predict(features_test)

In [21]:
#вычислим коэффициент детерминации предсказаний по отношению к целевому признаку
r2_transform = r2_score(target_test, predictions_t)

In [22]:
print('Коэффициент детерминации r2 на измененных данных составляет',
      r2_original)
print()
print('Коэффициент детерминации r2 на изначальных данных составляет',
      r2_transform)
print()
print('Разница в коэффициентах:', abs(r2_original - r2_transform))

Коэффициент детерминации r2 на измененных данных составляет 0.42494550286668

Коэффициент детерминации r2 на изначальных данных составляет 0.42681560948372166

Разница в коэффициентах: 0.001870106617041678


В результате мы получили очень близкие значения коэффициента `r2_score` для исходных и для измененных данных. Поставленная задача выполнена: мы изменили исходные данные согласно алгоритму, и получили на этих данных корректные предсказания модели.